In [ ]:
import os
from dotenv import load_dotenv
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker import (
    get_execution_role, image_uris, script_uris
)

In [ ]:
load_dotenv("env")

model_id = os.environ["IMAGE_CLASSIFICATION_MODEL_ID"]
model_version = os.environ["IMAGE_CLASSIFICATION_MODEL_VERSION"]
model_uri = os.environ["IMAGE_CLASSIFICATION_MODEL_URI"]

base_job_name = name_from_base(model_id.replace("tensorflow-", ""))
print(f"base_job_name: {base_job_name}")

In [ ]:
s3_input_path = os.path.join(os.environ["S3_INPUT_BUCKET"], os.environ["S3_PREFIX"], "")
s3_output_path = os.path.join(os.environ["S3_OUTPUT_BUCKET"], os.environ["S3_PREFIX"], model_id)

print(f"s3_input_path: {s3_input_path}")
print(f"s3_output_path: {s3_output_path}")

In [ ]:
# [Optional] Override default hyperparameters with custom values 
hyperparameters = {}
hyperparameters["augmentation"] = os.environ["AUGMENTATION"]
hyperparameters["augmentation_random_flip"] = os.environ["AUGMENTATION_RANDOM_FLIP"]
hyperparameters["augmentation_random_rotation"] = os.environ["AUGMENTATION_RANDOM_ROTATION"]
hyperparameters["augmentation_random_zoom"] = os.environ["AUGMENTATION_RANDOM_ZOOM"]
hyperparameters["batch_size"] = os.environ["BATCH_SIZE"]
hyperparameters["beta_1"] = os.environ["BETA_1"]
hyperparameters["beta_2"] = os.environ["BETA_2"]
hyperparameters["binary_mode"] = os.environ["BINARY_MODE"]
hyperparameters["dropout_rate"] = os.environ["DROPOUT_RATE"]
hyperparameters["early_stopping"] = os.environ["EARLY_STOPPING"]
hyperparameters["early_stopping_min_delta"] = os.environ["EARLY_STOPPING_MIN_DELTA"]
hyperparameters["early_stopping_patience"] = os.environ["EARLY_STOPPING_PATIENCE"]
hyperparameters["epochs"] = os.environ["EPOCHS"]
hyperparameters["epsilon"] = os.environ["EPSILON"]
hyperparameters["eval_metric"] = os.environ["EVAL_METRIC"]
hyperparameters["image_resize_interpolation"] = os.environ["IMAGE_RESIZE_INTERPOLATION"]
hyperparameters["initial_accumulator_value"] = os.environ["INITIAL_ACCUMULATOR_VALUE"]
hyperparameters["label_smoothing"] = os.environ["LABEL_SMOOTHING"]
hyperparameters["learning_rate"] = os.environ["LEARNING_RATE"]
hyperparameters["momentum"] = os.environ["MOMENTUM"]
hyperparameters["optimizer"] = os.environ["OPTIMIZER"]
hyperparameters["regularizers_l2"] = os.environ["REGULARIZERS_L2"]
hyperparameters["reinitialize_top_layer"] = os.environ["REINITIALIZE_TOP_LAYER"]
hyperparameters["rho"] = os.environ["RHO"]
hyperparameters["train_only_on_top_layer"] = os.environ["TRAIN_ONLY_ON_TOP_LAYER"]
hyperparameters

In [ ]:
# Create SageMaker Estimator instance
estimator = BaseEstimator(
    model_id=model_id,
    model_version=model_version,
    model_uri=model_uri,
    hyperparameters=hyperparameters,
    instance_type=os.environ["TRAINING_INSTANCE_TYPE"],
    instance_count=int(os.environ["TRAINING_INSTANCE_COUNT"]),
    max_run=int(os.environ["MAX_RUN"]),
    output_path=s3_output_path,
    base_job_name=base_job_name,
)

In [ ]:
%%time
estimator.fit({"training": s3_input_path}, logs="None")